In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-03-07T12:04:28.116499-05:00

Python implementation: CPython
Python version       : 3.8.10
IPython version      : 7.28.0

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 5.11.0-37-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 32
Architecture: 64bit



In [3]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
from dask.delayed import delayed
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
import logging

In [4]:
%watermark --iversions

dask   : 2022.9.2
numpy  : 1.23.5
pandas : 1.5.3
logging: 0.5.1.2
xgboost: 1.7.4
optuna : 2.10.1



In [5]:
!nvidia-smi -L

GPU 0: NVIDIA TITAN RTX (UUID: GPU-0eb32f58-b8d5-17c0-e952-f4ec26a9353f)
GPU 1: NVIDIA TITAN RTX (UUID: GPU-50aeb092-88f5-4e0b-7f73-32741666f319)


In [6]:
cluster = LocalCUDACluster(n_workers=2)
client = Client(cluster)

2023-03-07 12:04:40,327 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-03-07 12:04:40,327 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-03-07 12:04:40,336 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-03-07 12:04:40,336 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [7]:
!nvidia-smi

Tue Mar  7 12:04:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    On   | 00000000:09:00.0 Off |                  N/A |
| 41%   32C    P2    64W / 280W |    167MiB / 24220MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    On   | 00000000:41:00.0  On |                  N/A |
| 41%   

In [8]:
train_folds = []
val_folds = []
train_ys = []
val_ys = []

for i in range(5):
    print(f'Loading fold {i}')
    train_fold_d = delayed(pd.read_csv)(f'../input/xgtrain_5x_fold_{i}.csv.gz')
    train_fold = dd.from_delayed(train_fold_d)
    
    val_fold_d = delayed(pd.read_csv)(f'../input/xgval_fold_{i}.csv.gz')
    val_fold = dd.from_delayed(val_fold_d)
    
    
    train_y = train_fold['target']
    train_fold = train_fold[train_fold.columns.difference(['target'])]
    
    val_y = val_fold['target']
    val_fold = val_fold[val_fold.columns.difference(['target'])]
    
    train_folds.append(train_fold)
    val_folds.append(val_fold)
    
    train_ys.append(train_y)
    val_ys.append(val_y)

Loading fold 0
Loading fold 1
Loading fold 2
Loading fold 3
Loading fold 4


In [9]:
train_y

Dask Series Structure:
npartitions=1
    int64
      ...
Name: target, dtype: int64
Dask Name: getitem, 3 graph layers

In [10]:
train = pd.read_csv('../input/train.csv.gz')

target = train['label'].values
target

array([5, 0, 4, ..., 5, 6, 8])

In [11]:
train_oof = np.zeros((target.shape[0],))

num_round = 1000

def objective(trial):
        
    params = {
        'objective':'multi:softmax', 
        'num_class':10,
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),
        'random_state': 137,

    }
    
    #start_time = time()
    kf = StratifiedKFold(5, shuffle=True, random_state=1974)

    for i, (train_index, val_index) in enumerate(kf.split(train,target)):
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i], train_ys[i])
        dval = xgb.dask.DaskDMatrix(client, val_folds[i], val_ys[i])
        
        output = xgb.dask.train(client, params, dtrain, num_round)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        predictions = xgb.dask.predict(client, booster, dval)
        predictions = predictions.compute()
        train_oof[val_index] = predictions
        del dtrain, dval, output
        gc.collect()
        gc.collect()


    acc = accuracy_score(target, train_oof)
    
    return acc

In [12]:
train_oof.shape

(60000,)

In [13]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("optuna_xgb_output_0_5x.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.create_study(storage="sqlite:///xgb_optuna_mnist_0_5x.db", study_name="five_fold_optuna_xgb_0_5x", direction='maximize')

In [14]:
%%time
logger.info("Start optimization.")
study.optimize(objective, n_trials=3)

[12:10:07] task [xgboost.dask-1]:tcp://127.0.0.1:32965 got new rank 0
[12:13:28] task [xgboost.dask-0]:tcp://127.0.0.1:33351 got new rank 0
[12:16:47] task [xgboost.dask-1]:tcp://127.0.0.1:32965 got new rank 0
[12:20:08] task [xgboost.dask-0]:tcp://127.0.0.1:33351 got new rank 0
[12:23:29] task [xgboost.dask-1]:tcp://127.0.0.1:32965 got new rank 0
[12:26:52] task [xgboost.dask-0]:tcp://127.0.0.1:33351 got new rank 0
[12:29:33] task [xgboost.dask-1]:tcp://127.0.0.1:32965 got new rank 0
[12:32:16] task [xgboost.dask-0]:tcp://127.0.0.1:33351 got new rank 0
[12:34:57] task [xgboost.dask-1]:tcp://127.0.0.1:32965 got new rank 0
[12:37:42] task [xgboost.dask-0]:tcp://127.0.0.1:33351 got new rank 0
[12:40:24] task [xgboost.dask-1]:tcp://127.0.0.1:32965 got new rank 0
[12:43:42] task [xgboost.dask-0]:tcp://127.0.0.1:33351 got new rank 0
[12:46:55] task [xgboost.dask-1]:tcp://127.0.0.1:32965 got new rank 0
[12:50:11] task [xgboost.dask-0]:tcp://127.0.0.1:33351 got new rank 0
2023-03-07 12:53:17,

CPU times: user 1min 26s, sys: 30.8 s, total: 1min 57s
Wall time: 46min 35s


In [15]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,params_tree_method,state
0,0,0.978567,0.154950,0.708872,logloss,0.002564,0.078948,7,2,0.617537,gpu_hist,COMPLETE
1,1,0.973667,0.006109,0.356327,logloss,0.007613,0.099169,20,188,0.660917,gpu_hist,COMPLETE
2,2,0.944383,0.011867,0.973284,logloss,0.829396,0.004611,5,38,0.494260,gpu_hist,COMPLETE


In [16]:
%%time
study.optimize(objective, n_trials=5)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_0_5x.csv', index=False)
df.head()

[12:56:43] task [xgboost.dask-0]:tcp://127.0.0.1:33351 got new rank 0
2023-03-07 13:00:24,366 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
2023-03-07 13:00:24,538 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
[13:00:33] task [xgboost.dask-1]:tcp://127.0.0.1:32965 got new rank 0
2023-03-07 13:04:17,324 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
2023-03-07 13:04:17,494 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
[13:04:25] task [xgboost.dask-0]:tcp://127.0.0.1:33351 got new rank 0
2023-03-07 13:08:05,499 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
2023-03-07 13:08:05,673 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
[13:08:13]

CPU times: user 2min 23s, sys: 50.6 s, total: 3min 14s
Wall time: 1h 19min 1s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,params_tree_method,state
0,0,0.978567,0.154950,0.708872,logloss,0.002564,0.078948,7,2,0.617537,gpu_hist,COMPLETE
1,1,0.973667,0.006109,0.356327,logloss,0.007613,0.099169,20,188,0.660917,gpu_hist,COMPLETE
2,2,0.944383,0.011867,0.973284,logloss,0.829396,0.004611,5,38,0.494260,gpu_hist,COMPLETE
3,3,0.968100,0.183808,0.824116,logloss,5.695848,0.014858,11,197,0.750866,gpu_hist,COMPLETE
4,4,0.976317,0.006392,0.716599,logloss,2.086945,0.024511,13,38,0.492201,gpu_hist,COMPLETE


In [17]:
%%time
study.optimize(objective, n_trials=5)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_0_5x.csv', index=False)
df.head()

[14:15:44] task [xgboost.dask-1]:tcp://127.0.0.1:32965 got new rank 0
2023-03-07 14:18:44,770 - distributed.utils_perf - WARNING - full garbage collections took 88% CPU time recently (threshold: 10%)
2023-03-07 14:18:44,942 - distributed.utils_perf - WARNING - full garbage collections took 88% CPU time recently (threshold: 10%)
[14:18:53] task [xgboost.dask-0]:tcp://127.0.0.1:33351 got new rank 0
2023-03-07 14:21:51,428 - distributed.utils_perf - WARNING - full garbage collections took 88% CPU time recently (threshold: 10%)
2023-03-07 14:21:51,600 - distributed.utils_perf - WARNING - full garbage collections took 88% CPU time recently (threshold: 10%)
[14:21:59] task [xgboost.dask-1]:tcp://127.0.0.1:32965 got new rank 0
2023-03-07 14:25:02,772 - distributed.utils_perf - WARNING - full garbage collections took 88% CPU time recently (threshold: 10%)
2023-03-07 14:25:02,971 - distributed.utils_perf - WARNING - full garbage collections took 88% CPU time recently (threshold: 10%)
[14:25:11]

CPU times: user 2min 47s, sys: 55.4 s, total: 3min 43s
Wall time: 1h 32min 3s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,params_tree_method,state
0,0,0.978567,0.154950,0.708872,logloss,0.002564,0.078948,7,2,0.617537,gpu_hist,COMPLETE
1,1,0.973667,0.006109,0.356327,logloss,0.007613,0.099169,20,188,0.660917,gpu_hist,COMPLETE
2,2,0.944383,0.011867,0.973284,logloss,0.829396,0.004611,5,38,0.494260,gpu_hist,COMPLETE
3,3,0.968100,0.183808,0.824116,logloss,5.695848,0.014858,11,197,0.750866,gpu_hist,COMPLETE
4,4,0.976317,0.006392,0.716599,logloss,2.086945,0.024511,13,38,0.492201,gpu_hist,COMPLETE


In [18]:
df

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,params_tree_method,state
0,0,0.978567,0.154950,0.708872,logloss,0.002564,0.078948,7,2,0.617537,gpu_hist,COMPLETE
1,1,0.973667,0.006109,0.356327,logloss,0.007613,0.099169,20,188,0.660917,gpu_hist,COMPLETE
2,2,0.944383,0.011867,0.973284,logloss,0.829396,0.004611,5,38,0.494260,gpu_hist,COMPLETE
3,3,0.968100,0.183808,0.824116,logloss,5.695848,0.014858,11,197,0.750866,gpu_hist,COMPLETE
4,4,0.976317,0.006392,0.716599,logloss,2.086945,0.024511,13,38,0.492201,gpu_hist,COMPLETE
5,5,0.971000,0.001188,0.696028,logloss,1.880537,0.090995,3,269,0.829759,gpu_hist,COMPLETE
6,6,0.967417,0.100023,0.725954,logloss,1.141104,0.016900,7,206,0.650104,gpu_hist,COMPLETE
7,7,0.876933,0.143398,0.774907,logloss,0.460044,0.002794,3,93,0.913985,gpu_hist,COMPLETE
8,8,0.929500,0.045654,0.931134,logloss,2.639334,0.002675,5,10,0.433913,gpu_hist,COMPLETE
9,9,0.929683,1.525348,0.485128,logloss,0.954599,0.002688,20,262,0.492856,gpu_hist,COMPLETE


In [19]:
%%time
study.optimize(objective, n_trials=3)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_0_5x.csv', index=False)
df

[15:48:41] task [xgboost.dask-0]:tcp://127.0.0.1:33351 got new rank 0
2023-03-07 15:53:17,536 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
2023-03-07 15:53:17,714 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
[15:53:26] task [xgboost.dask-1]:tcp://127.0.0.1:32965 got new rank 0
2023-03-07 15:58:04,033 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
2023-03-07 15:58:04,210 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
[15:58:12] task [xgboost.dask-0]:tcp://127.0.0.1:33351 got new rank 0
2023-03-07 16:02:45,996 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
2023-03-07 16:02:46,164 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
[16:02:54]

CPU times: user 1min 50s, sys: 40.6 s, total: 2min 31s
Wall time: 1h 2min 46s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,params_tree_method,state
0,0,0.978567,0.154950,0.708872,logloss,0.002564,0.078948,7,2,0.617537,gpu_hist,COMPLETE
1,1,0.973667,0.006109,0.356327,logloss,0.007613,0.099169,20,188,0.660917,gpu_hist,COMPLETE
2,2,0.944383,0.011867,0.973284,logloss,0.829396,0.004611,5,38,0.494260,gpu_hist,COMPLETE
3,3,0.968100,0.183808,0.824116,logloss,5.695848,0.014858,11,197,0.750866,gpu_hist,COMPLETE
4,4,0.976317,0.006392,0.716599,logloss,2.086945,0.024511,13,38,0.492201,gpu_hist,COMPLETE
5,5,0.971000,0.001188,0.696028,logloss,1.880537,0.090995,3,269,0.829759,gpu_hist,COMPLETE
6,6,0.967417,0.100023,0.725954,logloss,1.141104,0.016900,7,206,0.650104,gpu_hist,COMPLETE
7,7,0.876933,0.143398,0.774907,logloss,0.460044,0.002794,3,93,0.913985,gpu_hist,COMPLETE
8,8,0.929500,0.045654,0.931134,logloss,2.639334,0.002675,5,10,0.433913,gpu_hist,COMPLETE
9,9,0.929683,1.525348,0.485128,logloss,0.954599,0.002688,20,262,0.492856,gpu_hist,COMPLETE
